In [1]:
#------------------------------------------------------------------------------+
#
#   Samar shaaban Haytamy
#   PSO service to select optimal web service  with Python
#   OCT, 2018
#
#------------------------------------------------------------------------------+

#--- IMPORT DEPENDENCIES ------------------------------------------------------+

from __future__ import division
import random
import time

from pandas import DataFrame
#from pandas import Series
#from pandas import concat
from pandas import read_csv
#from pandas import datetime
import numpy as np

import glob
import pandas as pd
from operator import itemgetter
import itertools

In [2]:
#read future QoS_provision data

files=glob.glob("csv_predictions/*.csv")
data=[]
providers_names=[]
for file in files:
    df=read_csv(file,header=0,names=['Av','Mx_rt','Mn_rt','Avg_rt','th'])
    providers_names.append(file)
    df.drop(df.columns[[1,2]], axis=1, inplace=True)
    data.append(df)

In [3]:
services=[]
for sublist in data:
    services.append(sublist.values)
print(services[0])

[[0.7080263  0.49288803 0.48884052]
 [0.6783135  0.5251736  0.52234197]
 [0.63854784 0.533731   0.54389113]
 [0.61676204 0.5480478  0.5601994 ]]


In [4]:
#split 100 services into 3 groups
class_service_1=services[0:33]
class_service_2=services[33:66]
class_service_3=services[66:100]

### read end user future preferences weights for each service

In [5]:
user_weights_df=read_csv('future user_behavior.csv',header=0,names=['Av','Rt','Th'])

In [6]:
W=list(user_weights_df.values)

In [7]:
#test cases
#class_service_1=[[0.21,0.8,0.35],[0.021,0.5,0.12]]
#class_service_2=[[0.821,0.08,0.5],[0.901,0.4,0.15]]
#class_service_3=[[0.621,0.25,0.22], [0.321,0.08,0.05]]    #???
#W=[[0.8,0.6,0.6],[0.1,0.3,0.8],[0.621,0.25,0.22], [0.321,0.08,0.05]]          #???

In [8]:
#W[0][1]
len(class_service_2)

33

In [9]:
#--- COST FUNCTION ------------------------------------------------------------+
# function we are attempting to optimize (minimize)

def func1(x):
    total_plan_value=0
    T_peroids=len(x)//3      #Time periods=4=total points or dimensions(x)/#classes which are 3 as assumed
 #   print("X=",x)
    for i in range(T_peroids):
        # assume the services are composed parallel aggragate QoS attributes values
        aggre_rt=max(class_service_1[x[3*i+0]][i,0],class_service_2[x[3*i+1]][i,0],class_service_3[x[3*i+2]][i,0])
        aggre_th=min(class_service_1[x[3*i+0]][i,1],class_service_2[x[3*i+1]][i,1],class_service_3[x[3*i+2]][i,1])
        aggre_cost=class_service_1[x[3*i+0]][i,2]+class_service_2[x[3*i+1]][i,2]+class_service_3[x[3*i+2]][i,2]
        value_plan_over_period_i=aggre_rt*W[i][0]+aggre_th*W[i][1]+aggre_cost*W[i][2]
        total_plan_value+=value_plan_over_period_i

    return total_plan_value

In [109]:


candidates_sols=[]
retrieved_sols=100
def add_sol(target_value,position):
    
    global candidates_sols
    candidates_sol=[]
    candidates_sols=sorted(candidates_sols, key=itemgetter(0))
    candidates_sols.reverse()
    
    #delete dubplicates
    candidates_sols=list(candidates_sols for candidates_sols,_ in itertools.groupby(candidates_sols))
    
    
    if len(candidates_sols)<retrieved_sols:
        candidates_sol.append(target_value)
        candidates_sol.extend(position)
        candidates_sols.append(candidates_sol)
        
        
    else:
        candidates_sol.append(target_value)
        candidates_sol.extend(position)
        candidates_sols[retrieved_sols-1]=candidates_sol
        

In [11]:
class Particle:
    def __init__(self,x0):
        self.position_i=[]                   # particle position
        self.velocity_i=[]                   # particle velocity
        self.pos_best_i=[]                   # best position individual
        self.local_best_fitness_value=-1     # best  individual fitness value
        self.fitness_value=-1                # current individual fitness value

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness using optimization function: costFunction
    def evaluate(self,costFunc):
        self.fitness_value=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.fitness_value > self.local_best_fitness_value or self.local_best_fitness_value==-1:
            self.pos_best_i=self.position_i
            self.local_best_fitness_value=self.fitness_value
            add_sol(self.fitness_value,self.position_i)

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.8      # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.randint(0,2)
            r2=random.randint(0,2)

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social
            

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]


            #want to make it integer number refers to services number (through bounds)==>update bounds
            #??? ok solved
            # adjust maximum position if necessary
            self.position_i[i]=int(self.position_i[i])
            if self.position_i[i]>bounds[i%3][1]:
                self.position_i[i]=bounds[i%3][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i%3][0]:
                self.position_i[i]=bounds[i%3][0]


In [12]:
pso_particles=[]

In [13]:
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        value_best_g=-1                   # best value for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,value_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].fitness_value > value_best_g or value_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    value_best_g=float(swarm[j].fitness_value)
                    #add_sol(value_best_g,pos_best_g)

            # cycle through swarm and update velocities and positioncostFunc
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        pos_best_g=np.array(pos_best_g)
        pos_best_g=pos_best_g.reshape(1,3)  #time period=4 , classes number=3
        print ('best global position:\n',pos_best_g)
        print ('\n \n best global target value:  ',value_best_g)
       
        #for j in range(0,num_particles):
            #print("\n \n position[",swarm[j].pos_best_i)
           # print("local_best_fitness_value[",swarm[j].local_best_fitness_value)
           

In [14]:
import random
import time
random.choice(range(len(class_service_1)))

#print(len(class_service_1)-1)

19

In [15]:
T_periods=1 #4
class_numbers=3
x0=[]
for i in range(T_periods):
    x0.append(random.choice(range(len(class_service_1))))
    x0.append(random.choice(range(len(class_service_2))))
    x0.append(random.choice(range(len(class_service_3))))

In [16]:
len(x0)

3

In [17]:
bounds=[(0, len(class_service_1)-1),(0,len(class_service_2)-1),(0,len(class_service_3)-1)]

In [18]:
bounds

[(0, 32), (0, 32), (0, 33)]

In [19]:
num_particle=20

In [20]:
#brute force service composition 
#for i in range (T)
total_plans=[]
for c_1 in range(len(class_service_1)):
    for c_2 in range(len(class_service_2)):
        for c_3 in range(len(class_service_3)):
            aggre_rt=max(class_service_1[c_1][0,0],class_service_2[c_2][0,0],class_service_3[c_3][0,0])
            aggre_th=min(class_service_1[c_1][0,1],class_service_2[c_2][0,1],class_service_3[c_3][0,1])
            aggre_cost=class_service_1[c_1][0,2]+class_service_2[c_2][0,2]+class_service_3[c_3][0,2]
            value_plan_over_period_0=aggre_rt*W[0][0]+aggre_th*W[0][1]+aggre_cost*W[0][2]
            total_plans.append([value_plan_over_period_0,c_1,c_2,c_3])

            

In [21]:

sorted_total_plans=sorted(total_plans, key=itemgetter(0))


In [22]:
sorted_total_plans.reverse()

brute force solutions

In [23]:
sorted_total_plans

[[2.7731554817159108, 31, 25, 22],
 [2.764138460670151, 31, 25, 17],
 [2.763819664905031, 31, 25, 7],
 [2.7592688314243268, 31, 19, 22],
 [2.756927867366828, 15, 16, 22],
 [2.7565813069530307, 31, 25, 15],
 [2.756204820767431, 31, 25, 2],
 [2.7516364387432386, 31, 10, 22],
 [2.7512627912926146, 31, 7, 22],
 [2.7503769561348546, 31, 1, 22],
 [2.750251810378567, 31, 19, 17],
 [2.749933014613447, 31, 19, 7],
 [2.749210753847468, 19, 16, 22],
 [2.747910846321068, 15, 16, 17],
 [2.747592050555948, 15, 16, 7],
 [2.7439024859370678, 16, 16, 22],
 [2.7426946566614467, 31, 19, 15],
 [2.7426194176974787, 31, 10, 17],
 [2.742318170475847, 31, 19, 2],
 [2.742300621932359, 31, 10, 7],
 [2.742245770246855, 31, 7, 17],
 [2.7419269744817347, 31, 7, 7],
 [2.741870041612231, 31, 25, 18],
 [2.7413599350890947, 31, 1, 17],
 [2.7411072893303476, 10, 16, 22],
 [2.7410411393239746, 31, 1, 7],
 [2.740353692603948, 15, 16, 15],
 [2.740193732801708, 19, 16, 17],
 [2.739977206418348, 15, 16, 2],
 [2.739963960109

In [24]:
len(total_plans)
len(sorted_total_plans)

37026

In [25]:
relevant_info=20
stp20=sorted_total_plans[0:relevant_info]

In [26]:
tp20=total_plans[0:retrieved_sols]

relevant brute force sols

In [27]:
relevant_BF_sol=[list(x) for x in set(tuple(x) for x in tp20).intersection(set(tuple(x) for x in stp20))]

In [28]:
relevant_BF_sol

[]

In [29]:
len(relevant_BF_sol)

0

PSo solutions calculation

In [257]:
PSO(func1,x0,bounds,num_particles=20,maxiter=500)

FINAL:
best global position:
 [[31 25 22]]

 
 best global target value:   2.7731554817159108


In [258]:
relevant_pso_sol=[list(x) for x in set(tuple(x) for x in candidates_sols).intersection(set(tuple(x) for x in stp20))]

In [259]:
relevant_pso_sol

[[2.756204820767431, 31, 25, 2],
 [2.747592050555948, 15, 16, 7],
 [2.742318170475847, 31, 19, 2],
 [2.7503769561348546, 31, 1, 22],
 [2.756927867366828, 15, 16, 22],
 [2.764138460670151, 31, 25, 17],
 [2.7731554817159108, 31, 25, 22],
 [2.7426194176974787, 31, 10, 17],
 [2.7592688314243268, 31, 19, 22],
 [2.7512627912926146, 31, 7, 22],
 [2.763819664905031, 31, 25, 7],
 [2.749210753847468, 19, 16, 22],
 [2.749933014613447, 31, 19, 7],
 [2.742300621932359, 31, 10, 7],
 [2.750251810378567, 31, 19, 17],
 [2.7516364387432386, 31, 10, 22],
 [2.7439024859370678, 16, 16, 22],
 [2.7565813069530307, 31, 25, 15],
 [2.7426946566614467, 31, 19, 15],
 [2.747910846321068, 15, 16, 17]]

In [260]:
len(relevant_pso_sol)

20

In [212]:
candidates_sols

[[2.7731554817159108, 31, 25, 22],
 [2.764138460670151, 31, 25, 17],
 [2.763819664905031, 31, 25, 7],
 [2.7592688314243268, 31, 19, 22],
 [2.756927867366828, 15, 16, 22],
 [2.7565813069530307, 31, 25, 15],
 [2.756204820767431, 31, 25, 2],
 [2.7516364387432386, 31, 10, 22],
 [2.7512627912926146, 31, 7, 22],
 [2.7503769561348546, 31, 1, 22],
 [2.750251810378567, 31, 19, 17],
 [2.749933014613447, 31, 19, 7],
 [2.749210753847468, 19, 16, 22],
 [2.747910846321068, 15, 16, 17],
 [2.747592050555948, 15, 16, 7],
 [2.7439024859370678, 16, 16, 22],
 [2.7426946566614467, 31, 19, 15],
 [2.742318170475847, 31, 19, 2],
 [2.742300621932359, 31, 10, 7],
 [2.742245770246855, 31, 7, 17],
 [2.7419269744817347, 31, 7, 7],
 [2.741870041612231, 31, 25, 18],
 [2.7413599350890947, 31, 1, 17],
 [2.7411072893303476, 10, 16, 22],
 [2.7410411393239746, 31, 1, 7],
 [2.740353692603948, 15, 16, 15],
 [2.740193732801708, 19, 16, 17],
 [2.739977206418348, 15, 16, 2],
 [2.7399639601092547, 31, 9, 22],
 [2.7398749370365